In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_extended_dataset_filtered.csv to final_extended_dataset_filtered.csv


In [ ]:
dataset = pd.read_csv("final_extended_dataset_filtered.csv")
dataset.head()

,Category,Subcategory,Type,Item_Name,Item_Variant,Sales_Type,Source,Sales_Count,Revenue
0,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,EZO QS,18,22000
1,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,POS,428,22000
2,Sales,BEVERAGES,COFFEE BASED,Americano (Hot),Americano (Hot),Ala Carte,GRAB,1,24000
3,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,EZO QS,14,25000
4,Sales,BEVERAGES,COFFEE BASED,Cappuccino,Cappuccino,Ala Carte,POS,162,25000




---


Seleksi feature dan target variable
kita nentuin variable apa yang kita ingin di karenakan pada testing kita mencari yang populer maka kita menggunakan variable ini


---



In [ ]:
X = dataset[['Item_Name', 'Sales_Count']]
y = dataset['Sales_Count']



---


Konversi Data Kategorikal Ke Dalam Numerikal
XGBoost tidak mampu berkerja dengan text/strings, jadi kita menggunakan Label Encoding untuk mengkonversikan



---




In [ ]:
label_encoders = {}
for col in X.columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

<ipython-input-80-d77dbfdb0b9d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-80-d77dbfdb0b9d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])




---


Spliting Data untuk training dan testing
-Ini supaya bisa nge-test akurasi model kepada data yang tidak terlihat


---



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




---
Training Model dengan 50 Epoch


> n_estimators=50,

XGBoost training menggunakan beberapa decision tree, setiap tree berikutnya akan memperbaiki tree sebelumnya













In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.1, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bylevel': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'colsample_bynode': [0.8, 1.0],
    'reg_lambda': [0.1, 1],


}

# Base XGBoost model with GPU support
xgb_model = xgb.XGBRegressor(
    n_estimators=50,
    objective="reg:absoluteerror",
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    random_state=42,
    verbosity=0
)


# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=2,
    verbose=1,
    n_jobs=-1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
xgb_model = grid_search.best_estimator_


Fitting 2 folds for each of 288 candidates, totalling 576 fits


Hasil
MAE lebih kecil lebih bagus, artinya jumlah error yang dilakukan oleh machine kecil

In [ ]:
y_pred = xgb_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"Test MAE: {mae:.4f}")

Test MAE: 2.6754


In [ ]:
# Add predictions to the test data
item_sales = X_test.copy()
item_sales['Predicted_Sales'] = y_pred

# Convert encoded Item_Name back to original names
item_sales['Item_Name'] = label_encoders['Item_Name'].inverse_transform(item_sales['Item_Name'])

# Find the most frequently predicted item
most_frequent_item = item_sales.groupby('Item_Name')['Predicted_Sales'].sum().idxmax()

print(f"The most frequently ordered item (predicted) is: {most_frequent_item}")

The most frequently ordered item (predicted) is: Nasi Goreng Ayam Spesial


NCF




---


Sama seperti sebelumnya harus di Encode terlebih dahulu


---



In [ ]:
item_encoder = LabelEncoder()
sales_count_encoder = LabelEncoder()

dataset['Item_ID'] = item_encoder.fit_transform(dataset['Item_Name'])
dataset['Sales_Count_ID'] = sales_count_encoder.fit_transform(dataset['Sales_Count'])

# Define features and target
X = dataset[['Item_ID', 'Sales_Count_ID']]
y = dataset['Sales_Count']



---


Definisikan kode

-memulai dari sumber

-mengkonversi dari ID kategorikal menjadi numerikal vektor

-menggabungkan kedua sumber lalui concat

-dan akhirnya membuat perdiksi


---






In [ ]:
item_input = Input(shape=(1,))
sales_count_input = Input(shape=(1,))


item_embedding = Embedding(input_dim=len(item_encoder.classes_), output_dim=10)(item_input)
sales_count_embedding = Embedding(input_dim=len(sales_count_encoder.classes_), output_dim=10)(sales_count_input)

item_vec = Flatten()(item_embedding)
sales_count_vec = Flatten()(sales_count_embedding)


concat = Concatenate()([item_vec, sales_count_vec])


fc1 = Dense(64, activation='relu')(concat)
fc2 = Dense(32, activation='relu')(fc1)
output = Dense(1)(fc2)

ncf_model = Model(inputs=[item_input, sales_count_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='mse')



---


Spltting lagi dan kemudian melakukan training dengan 32 batch dan learning speed 0.1


---



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
ncf_model.fit([X_train['Item_ID'], X_train['Sales_Count_ID']], y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 19361.6816 - val_loss: 10437.1641
Epoch 2/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 7770.7168 - val_loss: 4751.9912
Epoch 3/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3039.5454 - val_loss: 2689.1670
Epoch 4/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1432.4749 - val_loss: 2317.8730
Epoch 5/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 957.8068 - val_loss: 2053.4531
Epoch 6/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 496.2934 - val_loss: 1991.9104
Epoch 7/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 305.0760 - val_loss: 2050.3420
Epoch 8/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 233.8555 - val_loss: 2136.3369
Epoch 9/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 161.9162 - val_loss: 2294.3003
Epoch 10/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 105.5466 - val_loss: 2270.9404
Epoch 11/50
274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97.4823 - val_loss: 23

In [ ]:
predictions = ncf_model.predict([X_test['Item_ID'], X_test['Sales_Count_ID']])
most_ordered_item = item_encoder.inverse_transform([X_test.iloc[predictions.argmax()]['Item_ID']])[0]

print(f"Most frequently ordered item (predicted): {most_ordered_item}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Most frequently ordered item (predicted): Choco Banana


In [ ]:
y_pred=predictions
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE: {mae:.4f}')

MAE: 7.6873




---


XGBoost = 2.6754

NCF = 7.6873

Dari sini kita bisa liat bahwa Model AI yang memiliki akurasi tertinggi yaitu XGBoost


---

